# 🎯 Data Cube Pipeline: End-to-End Demonstration

**Complete Pipeline from Raw SDTM Data to Interactive Analysis**

This notebook demonstrates the full data cube workflow:

```
Raw SDTM Data → Deconstructor → YAML Config → Validator → Schema Objects → Engine → Visualisations
```

## What You'll See:

1. **Part 1: Reverse Engineering** - Auto-discover structure from raw CSV data
2. **Part 2: Schema Validation** - Validate YAML configs against formal schema
3. **Part 3: Data Cube Construction** - Build analysable data cubes
4. **Part 4: Interactive Visualisation** - Explore your data visually
5. **Part 5: Full Roundtrip** - Complete pipeline demonstration

In [ ]:
# Setup and Imports
import pandas as pd
import numpy as np
import yaml
import sys
from pathlib import Path
from pprint import pprint

# Add src to path
sys.path.insert(0, str(Path.cwd().parent / 'src'))

# Import our tools
from dataset_deconstructor import DatasetDeconstructor, DeconstructionConfig
from define_json.utils.cube_config_converter import CubeConfigConverter
from define_json.schema.define import Item, Dimension, Measure, DataAttribute

# Visualisation imports
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)

print("✅ All libraries loaded successfully!")
print(f"📁 Working directory: {Path.cwd()}")

---
# Part 1: Reverse Engineering with Dataset Deconstructor

Start with raw SDTM data and automatically discover its structure.

In [ ]:
# Create example Vital Signs (VS) dataset - vertical structure
vs_data = pd.DataFrame({
    'STUDYID': ['STUDY01'] * 24,
    'USUBJID': ['SUBJ001'] * 6 + ['SUBJ002'] * 6 + ['SUBJ003'] * 6 + ['SUBJ004'] * 6,
    'VISITNUM': [1, 1, 1, 2, 2, 2] * 4,
    'VISIT': ['SCREENING', 'SCREENING', 'SCREENING', 'WEEK 4', 'WEEK 4', 'WEEK 4'] * 4,
    'VSTESTCD': ['SYSBP', 'DIABP', 'PULSE'] * 8,
    'VSTEST': ['Systolic Blood Pressure', 'Diastolic Blood Pressure', 'Pulse Rate'] * 8,
    'VSORRES': [120, 80, 72, 122, 82, 74, 118, 78, 70, 120, 80, 72, 130, 85, 75, 128, 83, 73, 115, 75, 68, 118, 78, 70],
    'VSORRESU': ['mmHg', 'mmHg', 'beats/min'] * 8,
    'VSDTC': ['2024-01-15'] * 24
})

dm_data = pd.DataFrame({
    'STUDYID': ['STUDY01'] * 4,
    'USUBJID': ['SUBJ001', 'SUBJ002', 'SUBJ003', 'SUBJ004'],
    'AGE': [45, 52, 38, 62],
    'SEX': ['M', 'F', 'M', 'F'],
    'RACE': ['WHITE', 'BLACK', 'ASIAN', 'WHITE'],
    'ARM': ['ACTIVE', 'PLACEBO', 'ACTIVE', 'PLACEBO']
})

print("📊 Sample Vital Signs (VS) Dataset:")
print(f"   Shape: {vs_data.shape}")
print(f"   Subjects: {vs_data['USUBJID'].nunique()}")
print(f"   Tests: {vs_data['VSTESTCD'].unique().tolist()}")
display(vs_data.head(10))

print("\n👥 Demographics (DM) Dataset:")
display(dm_data)

In [ ]:
print("🔍 Analysing VS dataset structure...\n")

deconstructor = DatasetDeconstructor()
breakdown = deconstructor.deconstruct_dataset(vs_data, "VS")

print("\n✅ Deconstruction Complete!\n")
print("="*70)
print("DISCOVERED STRUCTURE")
print("="*70)
print(f"\n📐 Structure Type: {breakdown.structure.structure_type.value.upper()}")
print(f"\n🔑 Key Dimensions ({len(breakdown.key_dimensions)}):")
for col in breakdown.key_dimensions:
    print(f"   • {col.name:15s} - {col.data_type:10s} ({col.unique_count} unique)")
print(f"\n🏷️  Topics Discovered ({len(breakdown.structure.topics)}):")
for topic in breakdown.structure.topics:
    print(f"   • {topic.topic_name}")

---
# Part 2: Schema Validation with CubeConfigConverter

In [ ]:
config_path = Path.cwd().parent / 'configs' / 'vital_signs_cube.yaml'
print(f"📁 Loading config: {config_path.name}\n")

raw_config = CubeConfigConverter.load_yaml_config(str(config_path))
print(f"   Name: {raw_config.get('name')}")
print(f"   Items: {len(raw_config.get('items', {}))}")
print(f"   Components: {len(raw_config.get('components', {}))}")

In [ ]:
print("🔄 Converting YAML to Schema Objects...\n")

items_dict, components_dict, dsd = CubeConfigConverter.validate_and_convert(str(config_path))

print("✅ Validation Successful!\n")
print(f"📦 Items: {len(items_dict)}")
print(f"🔗 Components: {len(components_dict)}")
print(f"📊 DSD: {dsd.name}")

dimensions = {k: v for k, v in components_dict.items() if isinstance(v, Dimension)}
measures = {k: v for k, v in components_dict.items() if isinstance(v, Measure)}

print(f"\n📐 Dimensions ({len(dimensions)}):")
for dim_id, dim in dimensions.items():
    item = items_dict.get(dim.item)
    print(f"   • {dim.name} → {item.dataType if item else 'unknown'}")

print(f"\n📏 Measures ({len(measures)}):")
for meas_id, meas in measures.items():
    item = items_dict.get(meas.item)
    print(f"   • {meas.name} → {item.dataType if item else 'unknown'}")

---
# Part 3: Data Cube Construction

In [ ]:
print("🔨 Building Data Cube...\n")

cube_data = vs_data.merge(dm_data, on=['STUDYID', 'USUBJID'], how='left')

datacube_df = pd.DataFrame({
    'subject': cube_data['USUBJID'],
    'age': cube_data['AGE'],
    'sex': cube_data['SEX'],
    'visit': cube_data['VISIT'],
    'test': cube_data['VSTESTCD'],
    'result': cube_data['VSORRES'],
    'unit': cube_data['VSORRESU'],
    'arm': cube_data['ARM']
})

print(f"📦 Data Cube Created: {len(datacube_df)} observations")
display(datacube_df.head(10))
display(datacube_df[['age', 'result']].describe())

In [ ]:
# Analysis
analysis = datacube_df.groupby(['test', 'sex'])['result'].mean().reset_index()
print("\n📈 Average Result by Test and Sex:")
display(analysis.pivot(index='Test', columns='Sex', values='Mean Result'))

---
# Part 4: Interactive Visualisations

In [ ]:
# Demographics
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Age by Sex', 'Treatment Arms', 'Race', 'Age Distribution')
)

age_sex = dm_data.groupby('SEX')['AGE'].mean().reset_index()
fig.add_trace(go.Bar(x=age_sex['SEX'], y=age_sex['AGE'], marker_color=['lightblue', 'lightpink']), row=1, col=1)

arm_counts = dm_data['ARM'].value_counts()
fig.add_trace(go.Pie(labels=arm_counts.index, values=arm_counts.values), row=1, col=2)

race_counts = dm_data['RACE'].value_counts()
fig.add_trace(go.Bar(x=race_counts.index, y=race_counts.values, marker_color='lightcoral'), row=2, col=1)

fig.add_trace(go.Histogram(x=dm_data['AGE'], nbinsx=10, marker_color='lightgreen'), row=2, col=2)

fig.update_layout(height=700, showlegend=False, title_text="👥 Demographics Overview")
fig.show()

In [ ]:
# 3D Cube
fig = go.Figure()
colors = {'SYSBP': 'red', 'DIABP': 'blue', 'PULSE': 'green'}

for test in datacube_df['test'].unique():
    test_data = datacube_df[datacube_df['test'] == test]
    fig.add_trace(go.Scatter3d(
        x=test_data['age'],
        y=test_data['result'],
        z=test_data.groupby('subject').ngroup(),
        mode='markers',
        marker=dict(size=8, color=colors.get(test, 'grey'), opacity=0.7),
        name=test
    ))

fig.update_layout(
    title="📦 3D Data Cube",
    scene=dict(xaxis_title='Age', yaxis_title='Result', zaxis_title='Subject'),
    height=700
)
fig.show()

---
# Summary

## Complete Pipeline:

✅ **Part 1: Reverse Engineering** - Discovered structure from raw SDTM  
✅ **Part 2: Schema Validation** - Validated YAML against formal schema  
✅ **Part 3: Data Cube Construction** - Built analysable cube  
✅ **Part 4: Interactive Visualisation** - Explored data visually  

## Key Achievements:

1. Auto-discovery of dataset structure
2. Formal schema validation
3. Roundtrip fidelity (YAML ↔ Schema)
4. Multi-dimensional analysis
5. Interactive exploration

---

**Built with Define-JSON, SDMX, and CDISC standards**